# Proyecto de Ciencia de datos
**Proyecto**: Modelos Predictivos para la Estimación de Ventas de un Hotel
Resort en un Contexto de Crisis Económica       
**Estudiante**: Favio Rafael Guerra Calle                                                 
**Carrera**: Lic.Ingenieria Informatica


## **Fase 1**: Comprension del Negocio

Realizar predicciones de desempeño a futuro de las ventas de un hotel resort en un contexto de crisis económica, mediante el uso de modelos predictivos con el fin de apoyar la toma de decisiones estratégicas y evaluar la viabilidad del negocio.


## **Fase 2**: Comprension de los Datos

 **Hotel Resort**

Importacion de librerias inicial

In [ ]:
pip install prophet

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

Carga de datos

In [ ]:
df = pd.read_excel(r"../Data_Source/HotelResort.xlsx")

In [ ]:
print(df.columns)

In [ ]:
df = df.rename(columns={'Fecha Actual': 'Fecha', 'TOTAL FACTURA': 'TotalFactura'})

In [ ]:
df = df[['Fecha', 'TotalFactura']]

Convierte a formato fecha

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df['TotalFactura'] = pd.to_numeric(df['TotalFactura'], errors='coerce')

Elimina filas con valores nulos

In [ ]:
print(df.isnull().sum())
df.dropna(subset=['Fecha', 'TotalFactura'], inplace=True)

Media, mediana, desviación estándar

In [ ]:
print(df['TotalFactura'].describe())

In [ ]:
df.count()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df['TotalFactura'], bins=50, color='skyblue')
plt.title('Distribución del Total de Facturas')
plt.xlabel('Monto de la Factura')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df.index, df['TotalFactura'], marker='o', color='blue')
plt.title('Evolución del Total de Facturas a lo Largo del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('Monto de la Factura')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
Q1 = df['TotalFactura'].quantile(0.25)
Q3 = df['TotalFactura'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['TotalFactura'] < lower_bound) | (df['TotalFactura'] > upper_bound)]
num_outliers = len(outliers)

print(f"There are {num_outliers} outliers in TotalFactura.")

Agrupar las fechas iguales

## **Fase 3**: Preparacion de los Datos

 **Hotel Resort**

Agrupar las fechas iguales

In [ ]:
df = df.groupby('Fecha').sum()

In [ ]:
rango_completo = pd.date_range(
    start=df.index.min(),
    end=df.index.max(),
    freq='D'
)

In [ ]:
df = df.reindex(rango_completo)

In [ ]:
df['TotalFactura'] = df['TotalFactura'].fillna(0)

In [ ]:
df = df.resample('W').sum()

In [ ]:
Q1 = df['TotalFactura'].quantile(0.25)
Q3 = df['TotalFactura'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detectar outliers
outliers = df[(df['TotalFactura'] < lower_bound) | (df['TotalFactura'] > upper_bound)]

# Seleccionar los outliers significativos
outliers_top4 = outliers.reindex(outliers['TotalFactura'].abs().sort_values(ascending=False).index[:3])


In [ ]:
for i in range(len(outliers_top4)):
    idx = outliers_top4.index[i]
    # Opción 1: reemplazo por la media de los vecinos anteriores y posteriores
    prev_idx = df.index[df.index.get_loc(idx) - 1] if df.index.get_loc(idx) - 1 >= 0 else df.index[0]
    next_idx = df.index[df.index.get_loc(idx) + 1] if df.index.get_loc(idx) + 1 < len(df) else df.index[-1]

    prev_val = df['TotalFactura'].loc[prev_idx]
    next_val = df['TotalFactura'].loc[next_idx]
    suavizado = (prev_val + next_val) / 2

    df.at[idx, 'TotalFactura'] = suavizado

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df.index, df['TotalFactura'], marker='o', color='blue')
plt.title('Evolución del Total de Facturas a lo Largo del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('Monto de la Factura')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_prophet = df.reset_index().rename(columns={'Fecha': 'ds', 'TotalFactura': 'y'})
df_prophet = df_prophet.rename(columns={df_prophet.columns[0]: 'ds'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

print(df_prophet.tail())

In [ ]:
df_prophet = df_prophet.set_index('ds').ffill().reset_index()

**Exploracion de Datos**

In [ ]:
df.count()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df['TotalFactura'], bins=50, color='skyblue')
plt.title('Distribución del Total de Facturas')
plt.xlabel('Monto de la Factura')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df.sort_index(inplace=True)
df['TotalFactura'].resample('ME').sum().plot(title='Total de Facturas por Mes', figsize=(10, 5))
plt.ylabel('Monto Total')
plt.xlabel('Mes')
plt.grid(True)
plt.show()

In [ ]:
sns.boxplot(df['TotalFactura'])
plt.title('Boxplot del Total de Facturas')
plt.show()

Revizamos los outliers pero no se eliminara debido a la importancia de los mismos

In [ ]:
Q1 = df['TotalFactura'].quantile(0.25)
Q3 = df['TotalFactura'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['TotalFactura'] < lower_bound) | (df['TotalFactura'] > upper_bound)]
num_outliers = len(outliers)

print(f"HAY {num_outliers} outliers en TotalFactura.")

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df.index, df['TotalFactura'], marker='o', color='blue')
plt.title('Evolución del Total de Facturas a lo Largo del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('Monto de la Factura')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

Revision de autocorrelacion

In [ ]:
df_prophet = df_prophet.set_index('ds').ffill().reset_index()

Transformacion de Datos para estacionalidad

**Transformaciones Y Division de datos**

Division


In [ ]:
train_size = int(len(df_prophet) * 0.80)
train = df_prophet[:train_size]
test = df_prophet[train_size:]

In [ ]:
train_configurado = df_prophet[:train_size]
test_configurado = df_prophet[train_size:]
df_configurado = df_prophet.copy()

Variables exogenas

In [ ]:
festivos = pd.DataFrame({
    'holiday': 'CierreHotel',
    'ds': pd.to_datetime(['2022-05-21', '2022-07-22']),
    'lower_window': -30,  # Efecto comienza
    'upper_window': 10,   # Termina
})

Cuadratica

## **Fase 4**: Modelamiento

### **PROPHET BASICO**

In [ ]:
modelo = Prophet()
modelo.fit(train)

In [ ]:
future = test[['ds']].copy()
forecast = modelo.predict(future)

### **PROPHET CONFIGURADO**

In [ ]:
model_configurado = Prophet(
    growth='linear',
    seasonality_prior_scale=0.1,
    changepoint_prior_scale=0.1,
    holidays=festivos,
    holidays_prior_scale=15,
    weekly_seasonality=3,
    yearly_seasonality=True,
)
model_configurado.fit(train_configurado)

prediccion

In [ ]:
future_configurado = model_configurado.make_future_dataframe(periods=len(test), freq='W')
forecast_configurado = model_configurado.predict(future_configurado)

### **PROPHET HIBRIDO**

In [ ]:
modelo_hibrido = Prophet(
    growth='linear',
    seasonality_prior_scale=0.1,
    changepoint_prior_scale=0.1,
    holidays=festivos,
    holidays_prior_scale=15,
    #weekly_seasonality=True,
    weekly_seasonality=3,
    yearly_seasonality=True,)
modelo_hibrido.fit(train)

predicciones

In [ ]:
hibrido_future = modelo_hibrido.make_future_dataframe(periods=len(test), freq='W')
hibrido_forecast = modelo_hibrido.predict(hibrido_future)

In [ ]:
df_prophet = pd.merge(df_prophet, hibrido_forecast[['ds', 'yhat']], on='ds')
df_prophet['hibrido_yhat_prophet'] = df_prophet['yhat']
df_prophet['hibrido_residual'] = df_prophet['y'] - df_prophet['hibrido_yhat_prophet']

**FNN**

In [ ]:
df_prophet['hibrido_ds_ordinal'] = pd.to_datetime(df_prophet['ds']).map(pd.Timestamp.toordinal)
hibrido_X = df_prophet[['hibrido_ds_ordinal']].values
hibrido_y_residual = df_prophet['hibrido_residual'].values

normalizacion

In [ ]:
from sklearn.preprocessing import MinMaxScaler
hibrido_scaler = MinMaxScaler()
hibrido_X_scaled = hibrido_scaler.fit_transform(hibrido_X)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
hibrido_model_fnn = Sequential()
hibrido_model_fnn.add(Dense(128, input_dim=1, activation='relu'))
hibrido_model_fnn.add(BatchNormalization())
hibrido_model_fnn.add(Dropout(0.2))
hibrido_model_fnn.add(Dense(64, activation='relu'))
hibrido_model_fnn.add(BatchNormalization())
hibrido_model_fnn.add(Dense(1))
hibrido_model_fnn.compile(optimizer=Adam(learning_rate=0.005), loss='mse')

# Entrenamiento
hibrido_model_fnn.fit(hibrido_X_scaled[:train_size], hibrido_y_residual[:train_size], epochs=200, batch_size=32, verbose=0)

predicciones

In [ ]:
# Predicción de residuales
hibrido_y_residual_pred = hibrido_model_fnn.predict(hibrido_X_scaled).flatten()
df_prophet['hibrido_y_pred_final'] = df_prophet['hibrido_yhat_prophet'] + hibrido_y_residual_pred

**GP**

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C

# Paso 1: Calcular nuevos residuos después de Prophet + FNN
df_prophet['hibrido_residual_final'] = df_prophet['y'] - df_prophet['hibrido_y_pred_final']

# Paso 2: Crear y entrenar el Gaussian Process
hibrido_kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=7.0, length_scale_bounds=(1e-2, 1e3)) + WhiteKernel(noise_level=1)
hibrido_gp = GaussianProcessRegressor(kernel=hibrido_kernel, n_restarts_optimizer=10, alpha=0.1)
hibrido_gp.fit(hibrido_X_scaled, df_prophet['hibrido_residual_final'])

Prediccion y suma al modelo final

In [ ]:
hibrido_residual_gp_train = hibrido_gp.predict(hibrido_X_scaled)

df_prophet['hibrido_y_pred_full'] = df_prophet['hibrido_y_pred_final'] + hibrido_residual_gp_train

## **Fase 5**: Evaluacion

 **Hotel Resort**

### **Resultados PROPHET**

In [ ]:
pred = forecast['yhat'].values
real = test['y'].values

In [ ]:
rmse = np.sqrt(mean_squared_error(real, pred))
mae = mean_absolute_error(real, pred)
mape = np.mean(np.abs((real - pred) / real)) * 100

print(f"\nPROPHET - RMSE: {rmse:.2f}")
print(f"PROPHET - MAE: {mae:.2f}")
print(f"PROPHET - MAPE: {mape:.2f}%")

### **PROPHET CONFIGURADO**

In [ ]:
results_conf = pd.merge(df_configurado, forecast_configurado[['ds', 'yhat']], on='ds')
results_test = results_conf.iloc[train_size:]

In [ ]:
rmse = np.sqrt(mean_squared_error(results_test['y'], results_test['yhat']))
mae = mean_absolute_error(results_test['y'], results_test['yhat'])
mape = np.mean(np.abs((results_test['y'] - results_test['yhat']) / results_test['y'])) * 100

print(f"PROPHET (configurado) - RMSE: {rmse:.2f}")
print(f"PROPHET (configurado) - MAE: {mae:.2f}")
print(f"PROPHET (configurado) - MAPE: {mape:.2f}%")

### **Resultados PROPHET HIBRIDO**

In [ ]:
hibrido_test = df_prophet.iloc[train_size:].copy()
hibrido_rmse = np.sqrt(mean_squared_error(hibrido_test['y'], hibrido_test['hibrido_y_pred_final']))
hibrido_mae = mean_absolute_error(hibrido_test['y'], hibrido_test['hibrido_y_pred_final'])
hibrido_mape = np.mean(np.abs((hibrido_test['y'] - hibrido_test['hibrido_y_pred_final']) / hibrido_test['y'])) * 100

print(f"PROPHET (hibrido) - RMSE: {hibrido_rmse:.2f}")
print(f"PROPHET (hibrido) - MAE: {hibrido_mae:.2f}")
print(f"PROPHET (hibrido) - MAPE: {hibrido_mape:.2f}%")

**Resultados PROPHET HIBRIDO Completo**

In [ ]:
# Métricas finales en el conjunto de test
hibrido_test_full = df_prophet.iloc[train_size:].copy()

rmse_full = np.sqrt(mean_squared_error(hibrido_test_full['y'], hibrido_test_full['hibrido_y_pred_full']))
mae_full = mean_absolute_error(hibrido_test_full['y'], hibrido_test_full['hibrido_y_pred_full'])

print(f"RMSE (Modelo hibrido Final): {rmse_full:.2f}")
print(f"MAE (Modelo hibrido Final): {mae_full:.2f}")

## **Fase 6**: Despliegue

**Hotel Resort**

### **PROPHET**

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train['ds'], train['y'], label='Train (real)', color='blue')
plt.plot(test['ds'], real, label='Test (real)', color='orange')
plt.plot(test['ds'], pred, label='Predicción Prophet', color='green', linestyle='--')
plt.title('Predicción con Prophet')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### **PROPHET CONFIGURADO**

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(results_conf['ds'][:train_size], results_conf['y'][:train_size], label='Train (real)', color='blue')
plt.plot(results_conf['ds'][train_size:], results_conf['y'][train_size:], label='Test (real)', color='orange')
plt.plot(results_conf['ds'][train_size:], results_conf['yhat'][train_size:], label='Predicción Prophet', color='green', linestyle='--')
plt.xlabel('Fecha')
plt.ylabel('TotalFactura')
plt.title('Predicción Prophet - Separación Train/Test')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### **PROPHET HIBRIDO**

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_prophet['ds'][:train_size], df_prophet['y'][:train_size], label='Train Real', color='blue')
plt.plot(df_prophet['ds'][train_size:], df_prophet['y'][train_size:], label='Test Real', color='orange')
plt.plot(df_prophet['ds'][train_size:], df_prophet['hibrido_y_pred_final'][train_size:], label='Predicción Híbrida', color='green', linestyle='--')
plt.legend()
plt.grid(True)
plt.xlabel("Fecha")
plt.ylabel("TotalFactura")
plt.title("Predicción Híbrida Prophet + FNN")
plt.tight_layout()
plt.show()

Grafica Hibrido Completo

In [ ]:
plt.figure(figsize=(14, 6))

# Solo parte de entrenamiento
plt.plot(df_prophet['ds'][:train_size], df_prophet['y'][:train_size], label='Train Real', color='blue')
plt.plot(df_prophet['ds'][:train_size], df_prophet['hibrido_y_pred_full'][:train_size], label='Ajuste del Modelo (Hybrid)', linestyle='--', color='green')

plt.xlabel("Fecha")
plt.ylabel("TotalFactura")
plt.title("Ajuste del Modelo en el Conjunto de Entrenamiento - Prophet + FNN + GP")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_prophet['ds'][:train_size], df_prophet['y'][:train_size], label='Train Real', color='blue')
plt.plot(df_prophet['ds'][train_size:], df_prophet['y'][train_size:], label='Test Real', color='orange')
plt.plot(df_prophet['ds'][train_size:], df_prophet['hibrido_y_pred_full'][train_size:], label='Predicción Prophet+FNN+GP', color='green', linestyle='--')
plt.legend()
plt.grid(True)
plt.xlabel("Fecha")
plt.ylabel("TotalFactura")
plt.title("Predicción Híbrida (Prophet + FNN + Gaussian Process)")
plt.tight_layout()
plt.show()


In [ ]:
# Crear semanas futuras
future_hibrido = modelo_hibrido.make_future_dataframe(periods=84, freq='W')
forecast_hibrido = modelo_hibrido.predict(future_hibrido)

# Agregar predicción base de Prophet
forecast_hibrido['hibrido_yhat_prophet'] = forecast_hibrido['yhat']

In [ ]:
# Convertir fechas futuras a ordinal y normalizar
forecast_hibrido['ds_ordinal'] = pd.to_datetime(forecast_hibrido['ds']).map(pd.Timestamp.toordinal)
X_future = forecast_hibrido[['ds_ordinal']].values
X_future_scaled = hibrido_scaler.transform(X_future)  # usar el scaler entrenado


In [ ]:
# Residual FNN
residuals_fnn_future = hibrido_model_fnn.predict(X_future_scaled).flatten()

# Residual GP
residuals_gp_future = hibrido_gp.predict(X_future_scaled)

# Sumar predicciones: Prophet + FNN + GP
forecast_hibrido['hibrido_y_pred_final'] = forecast_hibrido['hibrido_yhat_prophet'] + residuals_fnn_future
forecast_hibrido['hibrido_y_pred_full'] = forecast_hibrido['hibrido_y_pred_final'] + residuals_gp_future


In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Histórico Real', color='blue')
plt.plot(df_prophet['ds'], df_prophet['hibrido_y_pred_full'], label='Predicción Histórica (Hybrid)', linestyle='--', color='green')
plt.plot(forecast_hibrido['ds'][-52:], forecast_hibrido['hibrido_y_pred_full'][-52:], label='Predicción Futura (Hybrid)', linestyle='--', color='red')
plt.axvline(x=df_prophet['ds'].iloc[-1], color='black', linestyle='dotted', label='Inicio predicción futura')
plt.xlabel("Fecha")
plt.ylabel("TotalFactura")
plt.title("Predicción Futura 1 Año - Modelo Híbrido (Prophet + FNN + GP)")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig(r"../Graficos/prediccion_Prophet.png", dpi=300)

plt.show()

In [ ]:
import pandas as pd

df_historico = df_prophet[['ds', 'y', 'hibrido_y_pred_full']].copy()

df_futuro = forecast_hibrido[['ds', 'hibrido_y_pred_full']].copy()

df_final = pd.concat([df_historico, df_futuro], ignore_index=True)

df_final.rename(columns={
    'ds': 'Fecha',
    'hibrido_y_pred_full': 'Prediccion_Hibrida'
}, inplace=True)

# Guardamos a CSV
#df_final.to_csv('prediccion_hibrida_historico_futuro.csv', index=False)
#print("CSV guardado como 'prediccion_hibrida_historico_futuro.csv'")


In [ ]:
df_futuro

In [ ]:
df_ultimosPrediccion = df_futuro.tail(51)
df_ultimosPrediccion

In [ ]:
df_ultimosPrediccion.to_csv(r"../Documentos/prediccion_Prophet_hibrida.csv", index=False)
print("CSV guardado como 'prediccion_hibrida_historico_futuro.csv'")